In [1]:
! export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt

In [2]:
%env CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt

env: CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt


In [3]:
import os
import boto3
from time import time
import xarray as xr

In [4]:
def _mkdir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory) 

In [5]:
_mkdir('./junkbox')

In [6]:
!ls

'00-build-xarrays-and -netcdfs.ipynb'	        Readme.md
 00-find-and-describe-wzells-data-order.ipynb   junkbox


In [7]:
def return_s3_list(working_bucket, prefix):
        aws_list = []
        s3 = boto3.resource('s3')
        bucket_name = working_bucket
        bucket = s3.Bucket(bucket_name)
        for obj in bucket.objects.filter(Prefix=prefix):
            obj_key = obj.key
            obj_key = working_bucket + '/' + obj_key
            aws_list.append((obj_key, obj.size))
        return aws_list

In [8]:
working_bucket = 'dev-et-data'
main_prefix = 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/'
year = 2003

target_prefix = main_prefix + str(year) 

the_list = return_s3_list(working_bucket, target_prefix)

In [9]:
the_list

[('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003001.tif',
  30377229),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003002.tif',
  11417794),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003003.tif',
  20252299),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003004.tif',
  2046065),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003005.tif',
  5080644),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003006.tif',
  6952156),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003007.tif',
  723933),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003008.tif',
  2973505),
 ('dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/dd_2003009

In [10]:
def create_s3_list_of_months(main_prefix, year, output_name='etasw_'):
    the_list = []
    for i in range(1,13):
        month = f'{i:02d}'
        file_object = main_prefix  + str(year) + '/' + output_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list

In [11]:
my_tifs = create_s3_list_of_months(main_prefix, year, output_name='etasw_')

In [12]:
my_tifs

['enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200301.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200302.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200303.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200304.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200305.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200306.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200307.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200308.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200309.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200310.tif',
 'enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200311.tif',
 'enduser/

In [13]:
def get_year_month(product, tif):
    fn = tif.split('/')[-1]
    fn = fn.replace(product,'')
    fn = fn.replace('.tif','')
    print(fn)
    return fn
 

In [14]:
def xr_build_cube_concat_ds(tif_list, product):

    start = time()
    my_da_list =[]
    year_month_list = []
    for tif in tif_list:
        tiffile = 's3://dev-et-data/' + tif
        print(tiffile)
        da = xr.open_rasterio(tiffile)
        #da = da.squeeze().drop(labels='band')
        #da.name=product
        my_da_list.append(da)
        tnow = time()
        elapsed = tnow - start
        print(tif, elapsed)
        year_month_list.append(get_year_month(product, tif))

    da = xr.concat(my_da_list, dim='band')
    da = da.rename({'band':'year_month'})
    da = da.assign_coords(year_month=year_month_list)
    DS = da.to_dataset(name=product)
    return(DS)

In [15]:
ds = xr_build_cube_concat_ds(my_tifs, 'etasw_')


s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200301.tif
enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200301.tif 0.5827491283416748
200301
s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200302.tif
enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200302.tif 0.9116730690002441
200302
s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200303.tif
enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200303.tif 1.2188243865966797
200303
s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200304.tif
enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200304.tif 1.5474023818969727
200304
s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/etasw_200305.tif
enduser/DelawareRiverBasin/Run09_13_2

In [16]:
ds

<xarray.Dataset>
Dimensions:     (x: 2885, y: 3846, year_month: 12)
Coordinates:
  * year_month  (year_month) <U6 '200301' '200302' ... '200311' '200312'
  * y           (y) float64 44.0 44.0 44.0 43.99 43.99 ... 36.01 36.0 36.0 36.0
  * x           (x) float64 -78.0 -78.0 -78.0 -77.99 ... -72.0 -72.0 -72.0
Data variables:
    etasw_      (year_month, y, x) float64 nan nan nan nan ... nan nan nan nan

In [17]:
help(ds.to_netcdf)

Help on method to_netcdf in module xarray.core.dataset:

to_netcdf(path=None, mode: str = 'w', format: str = None, group: str = None, engine: str = None, encoding: Mapping = None, unlimited_dims: Iterable[Hashable] = None, compute: bool = True, invalid_netcdf: bool = False) -> Union[bytes, ForwardRef('Delayed'), NoneType] method of xarray.core.dataset.Dataset instance
    Write dataset contents to a netCDF file.
    
    Parameters
    ----------
    path : str, Path or file-like, optional
        Path to which to save this dataset. File-like objects are only
        supported by the scipy engine. If no path is provided, this
        function returns the resulting netCDF file as bytes; in this case,
        we need to use scipy, which does not support netCDF version 4 (the
        default format becomes NETCDF3_64BIT).
    mode : {"w", "a"}, default: "w"
        Write ('w') or append ('a') mode. If mode='w', any existing file at
        this location will be overwritten. If mode='a', e

In [20]:
ds.to_netcdf('./junkbox/etasw_2003_ensemble.nc', engine='h5netcdf')

In [19]:
! pip list | grep netcdf

h5netcdf               0.8.1      


# NOTE

## Ran into this issue

https://github.com/pydata/xarray/issues/3374


- engine='h5netcdf'

- was the fix for me

In [22]:
! ls -lh ./junkbox

total 1016M
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 1016M Oct  5 22:56 etasw_2003_ensemble.nc
